In [3]:
import sys
import os
import re
import subprocess
import argparse
import numpy as np
import h5py
import glob
import matplotlib.pyplot as plt
import scipy.io

from scipy.interpolate import griddata
import scipy.interpolate as intp
from scipy.optimize import leastsq
from scipy.stats.stats import pearsonr

from pyrite import elevation_models
from pyrite import _utils as ut
from pykrige import OrdinaryKriging
from pykrige import variogram_models
#import matlab.engine # using matlab to estimate the variogram parameters
from mintpy.utils import ptime
###############################################################

model_dict = {'linear': elevation_models.linear_elevation_model,
                      'onn': elevation_models.onn_elevation_model,
                      'onn_linear': elevation_models.onn_linear_elevation_model,
                      'exp': elevation_models.exp_elevation_model,
                      'exp_linear': elevation_models.exp_linear_elevation_model}

residual_dict = {'linear': elevation_models.residuals_linear,
                      'onn': elevation_models.residuals_onn,
                      'onn_linear': elevation_models.residuals_onn_linear,
                      'exp': elevation_models.residuals_exp,
                      'exp_linear': elevation_models.residuals_exp_linear}

initial_dict = {'linear': elevation_models.initial_linear,
                      'onn': elevation_models.initial_onn,
                      'onn_linear': elevation_models.initial_onn_linear,
                      'exp': elevation_models.initial_exp,
                      'exp_linear': elevation_models.initial_exp_linear}

para_numb_dict = {'linear': 2,
                  'onn' : 3,
                  'onn_linear':4,
                  'exp':2,
                  'exp_linear':3}


variogram_dict = {'linear': variogram_models.linear_variogram_model,
                      'power': variogram_models.power_variogram_model,
                      'gaussian': variogram_models.gaussian_variogram_model,
                      'spherical': variogram_models.spherical_variogram_model,
                      'exponential': variogram_models.exponential_variogram_model,
                      'hole-effect': variogram_models.hole_effect_variogram_model}

def read_par_orb(slc_par):
    
    date0 = ut.read_gamma_par(slc_par,'read', 'date')
    date = date0.split(' ')[0] + date0.split(' ')[1] + date0.split(' ')[2]
    
    first_sar = ut.read_gamma_par(slc_par,'read', 'start_time')
    first_sar = str(float(first_sar.split('s')[0]))

    first_state = ut.read_gamma_par(slc_par,'read', 'time_of_first_state_vector')
    first_state = str(float(first_state.split('s')[0]))
    
    intv_state = ut.read_gamma_par(slc_par,'read', 'state_vector_interval')
    intv_state = str(float(intv_state.split('s')[0]))
    
    out0 = date + '_orbit0'
    out = date + '_orbit'
    
    if os.path.isfile(out0): 
        os.remove(out0)
    if os.path.isfile(out): 
        os.remove(out)
    
    with open(slc_par) as f:
        Lines = f.readlines()
    
    with open(out0, 'a') as fo:
        k0 = 0
        for Line in Lines:
            if 'state_vector_position' in Line:
                kk = float(first_state) + float(intv_state)*k0
                fo.write(str(kk - float(first_sar)) +' ' + Line)
                k0 = k0+1
                
    call_str = "awk '{print $1,$3,$4,$5}' " + out0 + " >" + out
    os.system(call_str)
    
    orb_data = ut.read_txt2array(out)
    #t_orb = Orb[:,0]
    #X_Orb = Orb[:,1]
    #Y_Orb = Orb[:,2]
    #Z_Orb = Orb[:,3]
    return orb_data

def remove_ramp(lat,lon,data):
    # mod = a*x + b*y + c*x*y
    lat = lat/180*np.pi
    lon = lon/180*np.pi  
    lon = lon*np.cos(lat) # to get isometrics coordinates
    
    p0 = [0.0001,0.0001,0.0001,0.0000001]
    plsq = leastsq(residual_trend,p0,args = (lat,lon,data))
    para = plsq[0]
    data_trend = data - func_trend(lat,lon,para)
    corr, _ = pearsonr(data, func_trend(lat,lon,para))
    return data_trend, para, corr

def func_trend(lat,lon,p):
    a0,b0,c0,d0 = p
    
    return a0 + b0*lat + c0*lon +d0*lat*lon

def residual_trend(p,lat,lon,y0):
    a0,b0,c0,d0 = p 
    return y0 - func_trend(lat,lon,p)

def func_trend_model(lat,lon,p):
    lat = lat/180*np.pi
    lon = lon/180*np.pi  
    lon = lon*np.cos(lat) # to get isometrics coordinates
    a0,b0,c0,d0 = p
    
    return a0 + b0*lat + c0*lon +d0*lat*lon

def OK_function(data0):
    OK,lat0,lon0,np = data0
    z0,s0 = OK.execute('points', lon0, lat0,n_closest_points= np,backend='loop')
    return z0,s0

def dist_weight_interp(data0):
    lat0,lon0,z0,lat1,lon1 = data0
    
    lat0 = np.asarray(lat0)
    lon0 = np.asarray(lon0)
    z0 = np.asarray(z0)
    
    if len(z0)==1:
        z0 = z0[0]
    nn = len(lat1)
    data_interp = np.zeros((nn,))
    weight_all = []
    for i in range(nn):
        dist0 = latlon2dis(lat0,lon0,lat1[i],lon1[i])
        weight0 = (1/dist0)**2
        if len(weight0) ==1:
            weight0 = weight0[0]
        weight = weight0/sum(weight0[:])
        data_interp[i] = sum(z0*weight)
        weight_all.append(weight)
    return data_interp,weight_all

def latlon2dis(lat1,lon1,lat2,lon2,R=6371):
    
    lat1 = np.array(lat1)*np.pi/180.0
    lat2 = np.array(lat2)*np.pi/180.0
    dlon = (lon1-lon2)*np.pi/180.0

    # Evaluate trigonometric functions that need to be evaluated more
    # than once:
    c1 = np.cos(lat1)
    s1 = np.sin(lat1)
    c2 = np.cos(lat2)
    s2 = np.sin(lat2)
    cd = np.cos(dlon)

    # This uses the arctan version of the great-circle distance function
    # from en.wikipedia.org/wiki/Great-circle_distance for increased
    # numerical stability.
    # Formula can be obtained from [2] combining eqns. (14)-(16)
    # for spherical geometry (f=0).

    dist =  R*np.arctan2(np.sqrt((c2*np.sin(dlon))**2 + (c1*s2-s1*c2*cd)**2), s1*s2+c1*c2*cd)

    return dist

######################################################################

date = 20151030
cdic = ut.initconst()
fname0 = '/Users/caoy0a/Desktop/Hawaii/ERA-5_N18_N21_W157_W153_20151030_16.grb'
fname0 = '/Users/caoy0a/Desktop/Hawaii/ERA-5_N15_N25_W160_W150_20200101_16.grb'
slc_par = '/Users/caoy0a/Desktop/Hawaii/20180105.slc.par'
geo_file = '/Users/caoy0a/Desktop/Hawaii/geometryRadar.h5'

datasetNames = ut.get_dataNames(geo_file)
lvls,latlist,lonlist,gph,tmp,vpr = ut.get_ecmwf('ERA5',fname0,cdic, humidity='Q')

meta = ut.read_attr(geo_file) 
if 'latitude' in datasetNames: 
    lats = ut.read_hdf5(geo_file,datasetName='latitude')[0]
    lons = ut.read_hdf5(geo_file,datasetName='longitude')[0]
else:
    lats,lons = ut.get_lat_lon(meta)  
heis = ut.read_hdf5(geo_file,datasetName='height')[0]    

attr = ut.read_attr(geo_file)
orb_data = read_par_orb(slc_par)
date0 = ut.read_gamma_par(slc_par,'read', 'date')
date = str(int(date0.split(' ')[0])) + str(int(date0.split(' ')[1])) + str(int(date0.split(' ')[2]))

start_sar = ut.read_gamma_par(slc_par,'read', 'start_time')
earth_R = ut.read_gamma_par(slc_par,'read', 'earth_radius_below_sensor')
end_sar = ut.read_gamma_par(slc_par,'read', 'end_time')
cent_time = ut.read_gamma_par(slc_par,'read', 'center_time')

attr['EARTH_RADIUS'] = str(float(earth_R.split('m')[0]))
attr['END_TIME'] = str(float(end_sar.split('s')[0]))
attr['START_TIME'] = str(float(start_sar.split('s')[0]))
attr['DATE'] = date
attr['CENTER_TIME'] = str(float(cent_time.split('s')[0]))

lvls,latlist,lonlist,gph,tmp,vpr = ut.get_ecmwf('ERA5',fname0 ,cdic, humidity='Q')
print(np.min(latlist[:]))
print(np.max(latlist[:]))

print(np.min(lonlist[:]))
print(np.max(lonlist[:]))

mean_lon = np.mean(lonlist.flatten())
if mean_lon > 180:    
    lonlist = lonlist - 360.0 # change to insar format lon [-180, 180]

lats0 = lats.flatten()
lons0 = lons.flatten()
mean_geoid = ut.get_geoid_point(np.nanmean(lats0),np.nanmean(lons0))
print('Average geoid height: ' + str(mean_geoid))
heis = heis + mean_geoid # geoid correct
heis0 = heis.flatten()

maxdem = max(heis0)
mindem = min(heis0)
if mindem < -200:
    mindem = -100.0

hh = heis0[~np.isnan(lats0)]
lala = lats0[~np.isnan(lats0)]
lolo = lons0[~np.isnan(lats0)]


sar_wet = np.zeros((heis.shape),dtype = np.float32)   
sar_wet0 = sar_wet.flatten()


sar_turb = np.zeros((heis.shape),dtype = np.float32)   
sar_turb0 = sar_wet.flatten()

sar_dry = np.zeros((heis.shape),dtype = np.float32)   
sar_dry0 = sar_dry.flatten()

hgtlvs = [ -200, 0, 200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000, 2200, 2400
          ,2600, 2800, 3000, 3200, 3400, 3600, 3800, 4000, 4200, 4400, 4600, 4800, 5000
          ,5500, 6000, 6500, 7000, 7500, 8000, 8500, 9000, 9500, 10000, 11000, 12000, 13000
          ,14000, 15000, 16000, 17000, 18000, 19000, 20000, 25000, 30000, 35000, 40000]
hgtlvs = np.asarray(hgtlvs)
Presi,Tempi,Vpri = ut.intP2H(lvls,hgtlvs,gph,tmp,vpr,cdic,verbose=False)
print(Presi.shape)
[DDry,DWet] = ut.PTV2del(Presi,Tempi,Vpri,hgtlvs,cdic)


lat_intp, lon_intp, los_intp = ut.get_LOS3D_coords(latlist,lonlist,hgtlvs, orb_data, attr)
# get los locations atmospheric paramters
##print('Start to calc LOS atmospheric parameters ...')
LosP,LosT,LosV = ut.get_LOS_parameters(latlist,lonlist,Presi,Tempi,Vpri,lat_intp, lon_intp,'kriging',15)
## calc los delays
##print('Start to calculate LOS delays ...')
ddrylos,dwetlos = ut.losPTV2del(LosP,LosT,LosV,los_intp ,cdic,verbose=False)

ddtot = ddrylos + dwetlos

print(dwetlos.shape)
print(hgtlvs.shape)


## interp grid delays
print('Start to interpolate delays ...')
dwet_intp,ddry_intp, lonvv, latvv, hgtuse = ut.pyrite_griddata_los(lon_intp,lat_intp,hgtlvs,ddrylos,dwetlos,attr, maxdem, mindem, 10,'kriging',15)

delay_tot = dwet_intp + ddry_intp

hgt = np.zeros((delay_tot.shape))
row,col,vert = delay_tot.shape

for i in range(vert):
    hgt[:,:,i] = hgtuse[i]

y00 = delay_tot.flatten()
x00 = hgt.flatten()    
    
y0 = y00[~np.isnan(y00)]
x0 = x00[~np.isnan(y00)]
    
### onn_linear model
model = 'onn_linear'
initial_function = initial_dict[model]
elevation_function = model_dict[model]
residual_function = residual_dict[model]

p0 = initial_function(x0,y0)
plsq = leastsq(residual_function,p0,args = (x0,y0))
print(plsq)
turb = y0 - elevation_function(plsq[0],x0)
corr_exp, _ = pearsonr(y0, elevation_function(plsq[0],x0))
y0_fit = elevation_function(plsq[0],x0)


sar_strat0 = elevation_function(plsq[0],heis)
print(corr_exp)
print(sar_strat0.shape)

turb_delay = delay_tot
turb_delay[~np.isnan(delay_tot)] = turb

print(turb_delay.shape)

#latv = latvv[:,0]
#lonv = lonvv[0,:]
#linearint_dry = intp.RegularGridInterpolator((latv[::-1], lonv, hgtuse), ddry_intp[::-1,:,:], method='linear', bounds_error=False, fill_value = 0.0)
#linearint_wet = intp.RegularGridInterpolator((latv[::-1], lonv, hgtuse), dwet_intp[::-1,:,:], method='linear', bounds_error=False, fill_value = 0.0)

ddtot_intp = ddry_intp + dwet_intp

linearint_turb = intp.RegularGridInterpolator((latv[::-1], lonv, hgtuse), turb_delay[::-1,:,:], method='linear', bounds_error=False, fill_value = 0.0)

sar_turb00  = linearint_turb(np.vstack((lala, lolo, hh)).T)
sar_turb0[~np.isnan(lats0)] = sar_turb00
sar_turb0 = sar_turb0.reshape(lats.shape)

sar_wet00  = linearint_wet(np.vstack((lala, lolo, hh)).T)
sar_wet0[~np.isnan(lats0)] = sar_wet00
sar_wet0 = sar_wet0.reshape(lats.shape)

sar_dry00  = linearint_dry(np.vstack((lala, lolo, hh)).T)
sar_dry0[~np.isnan(lats0)] = sar_dry00
sar_dry0 = sar_dry0.reshape(lats.shape)
sar_delay = sar_wet0 + sar_dry0

sar_turb0 = sar_turb0.reshape(lats.shape)
sar_strat0 = sar_strat0.reshape(lats.shape)

#turb = y0 - elevation_function(plsq[0],x0)
#corr_exp, _ = pearsonr(y0, elevation_function(plsq[0],x0))
#y0_fit = elevation_function(plsq[0],x0)
#ax.plot(x0, y0_fit, 'm.')



dict0 = dict()
#dict0['lat'] = lat_intp
#dict0['lon'] = lon_intp
#dict0['P0'] = Presi
#dict0['T0'] = Tempi
#dict0['V0'] = Vpri
#dict0['hgt_level'] = hgtlvs

#dict0['LOSP'] = LosP
#dict0['LOST'] = LosT
#dict0['LOSV'] = LosV
#dict0['dwet_los_sample'] = dwetlos
#dict0['ddry_los_sample'] = ddrylos

#dict0['lat_interp'] = latvv
#dict0['lon_interp'] = lonvv

#dict0['dwet_los_interp'] = dwet_intp
#dict0['ddry_los_interp'] = ddry_intp
#dict0['hgtuse'] = hgtuse

dict0['dry_zenith_sample'] = DDry
dict0['wet_zenith_sample'] = DWet
dict0['tot_zenith_sample'] = DDry + DWet


dict0['lon_sample_dense'] = lonv
dict0['hgt_sample_dense'] = hgtuse
dict0['lat_sample_dense'] = latv
dict0['delay_sample_dense'] = ddtot_intp
dict0['turb_sample_dense'] = turb_delay

dict0['sar_tot'] = sar_delay
dict0['sar_strat'] = sar_strat0
dict0['sar_turb'] = sar_turb0
heis1 = ut.read_hdf5(geo_file,datasetName='height')[0]   
dict0['dem'] = heis1
scipy.io.savemat('/Users/caoy0a/Desktop/Hawaii/test_zenith.mat', dict0)



fig = plt.figure()
ax = fig.add_subplot(212)
ax.plot(x0, y0, 'b.')
ax.yaxis.grid()
ax.plot(x0, y0_fit, 'k.')



















15.0
25.0
200.0
210.0
Average geoid height: 24.80805931184264
(41, 41, 51)


KeyboardInterrupt: 